In [ ]:
import numpy as np
import pandas as pd
import json
import dash
from dash import dcc, html, Input, Output
import time
import os 
import plotly.graph_objects as go
from dash.dependencies import Input, Output

import plotly.express as px
import matplotlib.pyplot as plt

from cleanup.cleanup_functions import clean_numeros
from cleanup.cleanup_functions import classify_id

# 📂 Données nécessaires pour exécuter ce notebook

Pour faire fonctionner ce notebook, vous devez placer plusieurs fichiers **.csv** dans le dossier `data/`. Certains fichiers sont volumineux et doivent être téléchargés manuellement.

## 📥 Fichiers requis

### 1️⃣ Fichiers à télécharger manuellement :
- **`StockEtablissement_utf8.csv`** (⚠️ ~8 Go)  
  📥 Disponible ici : [Base SIRENE des entreprises et de leurs établissements (SIREN/SIRET)](https://www.data.gouv.fr/fr/datasets/base-sirene-des-entreprises-et-de-leurs-etablissements-siren-siret/)  

- **`20230823-communes-departement-region.csv`**  
  📥 Disponible ici : [Communes de France - Base des codes postaux](https://www.data.gouv.fr/fr/datasets/communes-de-france-base-des-codes-postaux/)  

### 2️⃣ Autres fichiers attendus dans `data/` :
Ces fichiers sont utilisés dans le notebook, assurez-vous qu'ils sont bien présents :
- **`all_communities_data.csv`**  
- **`marches_publics_normalized_data.csv`**  
- **`selected_communities_data.csv`**  
- **`subventions_normalized_data.csv`**  

⚠️ **Remarque** : Ces fichiers sont exclus du dépôt Git via `.gitignore` pour éviter d'ajouter des fichiers trop lourds.

## 📌 Instructions :
1. **Téléchargez** les fichiers manuellement si nécessaire.  
2. **Placez-les** dans le dossier `data/` à la racine du projet.  
3. **Vérifiez** que les chemins sont corrects avant d'exécuter le notebook.

Bonne analyse ! 🚀


In [ ]:
path_to_data = 'data' #path to data

## READ DATA

Statistique des Regions:
Regroupement de ces data_set: 


In [ ]:
Data_region_statistique = pd.read_csv(path_to_data+'/INSEE_Region.csv')

In [ ]:
Data_region_statistique_mean = Data_region_statistique[['cog','year','Total',
       'Président du conseil régional', 'Positionnement parti',
       'Parti politique', 'Chomage moyenne anneé', 'De 20 à 64 ans',
       '65 ans et plus','Moins de 20 ans']]

In [38]:
file_path = 'data/Subvention_normalized_data_with_NAF.csv'

if os.path.exists(file_path):
    subventions_normalized_data_with_NAF = pd.read_csv(path_to_data+'/Subvention_normalized_data_with_NAF.csv', on_bad_lines='skip', sep=';')
    print(f"Le fichier {file_path} Le fichier {file_path} existe déjà. On passe directement au Dashboard global Region.")
else:
    print(f"Le fichier {file_path} n'existe pas, Clean Siret et Add naf doivent etre exécutées.")


Le fichier data/Subvention_normalized_data_with_NAF.csv Le fichier data/Subvention_normalized_data_with_NAF.csv existe déjà. On passe directement au Dashboard global Region.


In [37]:
subventions_normalized_data = pd.read_csv(path_to_data+'/subventions_normalized_data.csv', on_bad_lines='skip', sep=';')
selected_communities_data = pd.read_csv(path_to_data+'/selected_communities_data.csv', sep=';')
marches_publics_normalized_data = pd.read_csv(path_to_data+'/marches_publics_normalized_data.csv', sep=';')
all_communities_data = pd.read_csv(path_to_data+'/all_communities_data.csv', sep=';')


/var/folders/nn/3glhgr3j7hq9dt7rpd6sks840000gn/T/ipykernel_473/593674888.py:1: DtypeWarning:

Columns (1,2,3,4,6,7,11,12,13,14,16,18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.

/var/folders/nn/3glhgr3j7hq9dt7rpd6sks840000gn/T/ipykernel_473/593674888.py:3: DtypeWarning:

Columns (1,2,7,8,17) have mixed types. Specify dtype option on import or set low_memory=False.

/var/folders/nn/3glhgr3j7hq9dt7rpd6sks840000gn/T/ipykernel_473/593674888.py:4: DtypeWarning:

Columns (4,5,8) have mixed types. Specify dtype option on import or set low_memory=False.



Le fichier data/Subvention_normalized_data_with_NAF.csv Le fichier data/Subvention_normalized_data_with_NAF.csv existe déjà. On passe directement au Dashboard global Region.


In [ ]:
communesdepartementregion = pd.read_csv(path_to_data+'/20230823-communes-departement-region.csv')

## Clean Siret

In [ ]:

subventions_normalized_data['idbeneficiaire'] = subventions_normalized_data['idbeneficiaire'].apply(clean_numeros)
subventions_normalized_data['nombeneficiaire'] = subventions_normalized_data['nombeneficiaire'].apply(clean_numeros)

subventions_normalized_data[['siret_bene', 'siren_bene', 'nombeneficiaire_new','switch']] = subventions_normalized_data.apply(
    lambda row: pd.Series(classify_id(row['idbeneficiaire'], row['nombeneficiaire'])), axis=1
)
subventions_normalized_data['dateconvention'] = pd.to_datetime(subventions_normalized_data['dateconvention'], errors='coerce')
subventions_normalized_data['year'] = subventions_normalized_data['dateconvention'].dt.year
# subventation = subventions_normalized_data.loc[~(subventions_normalized_data['montant'].isna())]
# subventation = subventation.loc[subventation['montant']!=0]
subventions_normalized_data = subventions_normalized_data.rename(columns={'siren': 'siren_attribuant'})

### Add code NAF

In [ ]:
list_siret_bene = list(set(subventions_normalized_data['siret_bene']))
list_siret_bene =  np.array([np.nan if pd.isna(i) else np.int64(i) for i in list_siret_bene])
subventions_normalized_data = subventions_normalized_data.merge(selected_communities_data[['nom','type','cog', 'population','longitude',
       'latitude']], 
                     left_on=['nom','type'], 
                     right_on=['nom','type'], 
                     how='left')

data_naf = pd.read_excel("data/int_courts_naf_rev_2.xls")
data_naf = data_naf.loc[~(data_naf['Code'].isna())]
data_naf.columns = ['ligne', 'Code', 'Intitulés de la NAF rév. 2, version finale',
       'Intitulés NAF rév. 2, \nen 65 caractères',
       'Intitulés NAF rév. 2, \nen 40 caractères']

In [ ]:

chunksize = 10**5
data_siret = pd.DataFrame({})

for chunk in pd.read_csv(path_to_data+'/StockEtablissement_utf8.csv', chunksize=chunksize, low_memory=False):

    data_siret = pd.concat([data_siret,chunk.loc[chunk['siret'].isin(list_siret_bene)]])
# À la fin de la boucle, 'data_siret' contient toutes les lignes du fichier CSV 'StockEtablissement_utf8.csv'
# où la colonne 'siret' correspond à une valeur de 'list_siret_bene'

In [ ]:
data_siret["siret"] = data_siret["siret"].astype(str)
index_to_put = ['siren','siret','nic','trancheEffectifsEtablissement','activitePrincipaleEtablissement','activitePrincipaleRegistreMetiersEtablissement','codeCommuneEtablissement','nomenclatureActivitePrincipaleEtablissement']

df_merged = subventions_normalized_data.merge(data_siret.loc[:,index_to_put], left_on="siret_bene", right_on="siret", how="left")

df_merged['NAF_subsector'] = df_merged['activitePrincipaleEtablissement'].str.extract(r'(\d{2})')

df_merged = df_merged.merge(
    data_naf[['Code', 'Intitulés de la NAF rév. 2, version finale']], 
    left_on="NAF_subsector", right_on="Code", how="left",
    suffixes=('', '_drop')
).rename(columns={'Intitulés de la NAF rév. 2, version finale': 'Naf_subsector_name'})

df_merged = df_merged.merge(
    data_naf[['Code', 'Intitulés NAF rév. 2, \nen 40 caractères']], 
    left_on="activitePrincipaleEtablissement", right_on="Code", how="left",
    suffixes=('', '_drop')  
).rename(columns={'Intitulés NAF rév. 2, \nen 40 caractères': 'Naf_subsubsector_name'})

df_merged = df_merged.drop(columns=['NAF_subsector', 'Code', 'Code_drop'], errors='ignore')



In [ ]:

data_naf_sector = data_naf[['Code', 'Intitulés NAF rév. 2, \nen 40 caractères']]

# Initialisation des variables
sections = []
codes = []
titles = []
current_codes = []
current_section = None
current_title = None

# Parcours du dataframe pour regrouper les codes par section
for _, row in data_naf_sector.iterrows():
    if 'SECTION' in row['Code']:  # Nouvelle section détectée
        if current_codes:  
            codes.append(current_codes)
            sections.append(current_section)
            titles.append(current_title)

        current_section = row['Code']
        current_title = row['Intitulés NAF rév. 2, \nen 40 caractères']
        current_codes = []
    else:
        current_codes.append(row['Code'])

if current_codes:
    codes.append(current_codes)
    sections.append(current_section)
    titles.append(current_title)

df_naf_section = pd.DataFrame({
    'SECTION': sections,
    'Intitulés NAF rév. 2, \nen 40 caractères': titles,
    'Code': codes
})

naf_section_mapping = {code: title for codes_list, title in zip(df_naf_section['Code'], df_naf_section['Intitulés NAF rév. 2, \nen 40 caractères']) for code in codes_list}

df_merged['section NAF'] = df_merged['activitePrincipaleEtablissement'].map(naf_section_mapping)
subventions_normalized_data_with_NAF = df_merged.drop(columns=['Unnamed: 0'], errors='ignore')

In [ ]:
#df_merged_subvention.to_csv('data/Subvention_new_data.csv')

## DASHBOARD REGION GLOBAL

In [ ]:
data_reg = subventions_normalized_data_with_NAF.loc[subventions_normalized_data_with_NAF['type'] == 'REG']

J'enleve si il n'y a pas d'année

In [ ]:
data_reg = data_reg.dropna(subset=['year'])

In [ ]:
data_reg['year'] = data_reg['year'].astype('Int64')
data_reg['cog'] = data_reg['cog'].astype('Int64')

data_reg_group = data_reg.groupby(['year','nom','population','longitude','latitude','cog']).sum().reset_index()
data_reg_group['year'] = data_reg_group['year'].astype(int)
data_reg_group = data_reg_group.merge(Data_region_statistique_mean, on = ['year','cog'])

In [ ]:

# Fonction pour formater les montants
def custom_format(val):
    if val >= 1_000_000_000:
        return f"{val / 1_000_000_000:.1f} Md"
    elif val >= 1_000_000:
        return f"{val / 1_000_000:.1f} Mi"
    else:
        return f"{val:.0f}"

data_reg_group['montant_formatted'] = data_reg_group['montant'].apply(custom_format)

# Charger le GeoJSON des régions françaises
geojson_path = path_to_data + "/regions.geojson"
with open(geojson_path, "r", encoding="utf-8") as file:
    geojson_regions = json.load(file)

fig = px.choropleth_mapbox(
    data_reg_group[data_reg_group['year'] == 2014],  # Filtrer les données pour l'année 2014
    geojson=geojson_regions,
    locations="cog",
    featureidkey="properties.code",
    color="montant",
    color_continuous_scale="Viridis",
    hover_name="nom",
    hover_data=["montant_formatted"],
    mapbox_style="carto-positron",
    zoom=4.5,
    center={"lat": 46.603354, "lon": 1.888334},
    opacity=0.7,
    title="Carte des Subventions par Région (2014)",
)

buttons = [
    {
        "args": [{"visible": [year == selected_year for selected_year in sorted(data_reg_group['year'].unique())]}, 
                 {"title": f"Carte des Subventions par Région ({year})"}],
        "label": f"{year}",
        "method": "update"
    }
    for year in sorted(data_reg_group['year'].unique())
]


fig.update_layout(
        coloraxis_colorbar=dict(
        title="Montant des Subventions (€)",
        tickvals=[0, 1000000000, 5000000000, 10000000000],  # Les valeurs à afficher
        ticktext=["0", "1 Md", "5 Md","10 Md"],  # Texte des ticks
    ),
    updatemenus=[
        {
            "buttons": buttons,
            "direction": "down",
            "showactive": True,
            "active": 0,
            "x": 0.17,
            "xanchor": "left",
            "y": 0.98,
            "yanchor": "top"
        }
    ],
    geo=dict(
        showcoastlines=True,
        coastlinecolor="Black",
        projection_type="mercator",
        showland=True,
        landcolor="lightgray",
    ),
    width=800,
    height=600
)

for year in sorted(data_reg_group['year'].unique()):
    fig.add_trace(
        px.choropleth_mapbox(
            data_reg_group[data_reg_group['year'] == year],
            geojson=geojson_regions,
            locations="cog",
            featureidkey="properties.code",
            color="montant",
            color_continuous_scale="Viridis",
            hover_name="nom",
            hover_data=["montant_formatted"],
            mapbox_style="carto-positron",
            opacity=0.7,
        ).data[0]
    )

fig.show()
# fig.write_image("carte_subventions_region.png")
# fig.write_html("carte_subventions_region.html")

### Carte Sub with data on region

In [ ]:

# Création de l'application Dash
app = dash.Dash(__name__)


geojson_path = path_to_data + "/regions.geojson"
with open(geojson_path, "r", encoding="utf-8") as file:
    geojson_regions = json.load(file)


# Fonction pour formater les montants
def custom_format(val):
    if val >= 1_000_000_000:
        return f"{val / 1_000_000_000:.1f} Md"
    elif val >= 1_000_000:
        return f"{val / 1_000_000:.1f} Mi"
    else:
        return f"{val:.0f}"


data_reg_group['montant_formatted'] = data_reg_group['montant'].apply(custom_format)


def create_map(year):
    filtered_data = data_reg_group[data_reg_group['year'] == year]
    
    fig = px.choropleth_mapbox(
        filtered_data, 
        geojson=geojson_regions,
        locations="cog",
        featureidkey="properties.code",
        color="montant",
        color_continuous_scale="Viridis",
        hover_name="nom",
        hover_data=["montant_formatted"],
        mapbox_style="carto-positron",
        zoom=4.5,
        center={"lat": 46.603354, "lon": 1.888334},
        opacity=0.7,
    )
    
    fig.update_layout(
        title=f"Carte des Subventions par Région ({year})",
        coloraxis_colorbar=dict(
            title="Montant des Subventions (€)",
            tickvals=[0, 1000000000, 5000000000, 10000000000],
            ticktext=["0", "1 Md", "5 Md", "10 Md"]
        ),
        geo=dict(
            showcoastlines=True,
            coastlinecolor="Black",
            projection_type="mercator",
            showland=True,
            landcolor="lightgray",
        ),
        width=800,
        height=600,
        margin={"r":0,"t":50,"l":0,"b":0}  # Ajuster les marges pour l'affichage
    )
    
    return fig


app.layout = html.Div([
    html.Div([
        dcc.Dropdown(
            id='year-dropdown',
            options=[{'label': str(year), 'value': year} for year in sorted(data_reg_group['year'].unique())],
            value=2014,
            style={'width': '50%'}
        ),
        dcc.Graph(
            id='choropleth-map',
            figure=create_map(2014)  
        ),
    ], style={'width': '100%', 'display': 'inline-block'}),  

    html.Div([
        html.H3("Détails de la région"),
        html.Div(id='region-details', style={'white-space': 'pre-line'})  
    ], style={'width': '100%', 'display': 'inline-block', 'padding-top': '20px'})  
])

@app.callback(
    [Output('choropleth-map', 'figure'),
     Output('region-details', 'children')],
    [Input('year-dropdown', 'value'),
     Input('choropleth-map', 'clickData')]
)
def update_map(year, clickData):
    fig = create_map(year)  
    
    if clickData is None:
        return fig, "Cliquez sur une région pour afficher les détails."
    
    region_code = clickData['points'][0]['location']
    
    region_data = data_reg_group[(data_reg_group['cog'] == region_code) & (data_reg_group['year'] == year)].iloc[0]
    
    return fig, html.Div([
        html.P(f"Nom de la région: {region_data['nom']}"),
        html.P(f"Président du conseil régional: {region_data['Président du conseil régional']}"),
        html.P(f"Parti politique: {region_data['Parti politique']}"),
        html.P(f"Chômage moyen: {region_data['Chomage moyenne anneé']:.2f}%"),
        html.P(f"Moins de 20 ans: {region_data['Moins de 20 ans']}"),
        html.P(f"De 20 à 64 ans: {region_data['De 20 à 64 ans']}"),
        html.P(f"65 ans et plus: {region_data['65 ans et plus']}"),
        html.P(f"Total population: {region_data['Total']}")
    ])

if __name__ == '__main__':
    app.run_server(debug=True)


# UNE SEULE REGION 

In [ ]:
data_reg['cog'].value_counts()

In [ ]:
COG = 53

In [ ]:
region = data_reg.loc[data_reg['cog'] == COG,'nomattribuant'].unique()[0]

In [ ]:
data_region = data_reg[data_reg['cog'] == COG]

In [ ]:
subventions_beneficiaire_region = data_region.groupby(['year', 'nombeneficiaire'])['montant'].sum().reset_index()

## Subventation per year

In [ ]:
subventions_region_per_year = data_region.groupby(['year'])['montant'].sum().reset_index()

In [ ]:
fig = px.bar(subventions_region_per_year, x="year", y="montant", barmode="group",
              title=f"Montant attribué par la {region} et par année")

fig.show()

## Beneficiaire

In [ ]:
subventions_beneficiaire_region_sorted = subventions_beneficiaire_region.sort_values(by=['year','montant'], ascending=[True, False])

In [ ]:

top_beneficiaries = pd.DataFrame()

# Parcourir chaque région et sélectionner les 10 plus gros bénéficiaires
for year in subventions_beneficiaire_region_sorted['year'].unique():
    year_data = subventions_beneficiaire_region_sorted[subventions_beneficiaire_region_sorted['year'] == year]
    top_10_region = year_data.sort_values(by='montant', ascending=False).head(10)
    top_beneficiaries = pd.concat([top_beneficiaries, top_10_region])

for year in [2019,2020,2021,2022,2023,2024]:
    year_data = top_beneficiaries[top_beneficiaries['year'] == year]

    fig = px.bar(year_data, 
                 x='nombeneficiaire', 
                 y='montant', 
                 title=f"Top 10 des bénéficiaires - {year}",
                 labels={'nombeneficiaire': 'Bénéficiaire', 'montant': 'Montant total'},
                 color='nombeneficiaire',  # Chaque bénéficiaire a une couleur différente
                 height=600)


    fig.update_layout(  xaxis = go.XAxis(
        title = 'Bénéficiaire',
        showticklabels=False),)

   # fig.write_image(f"Top_beneficiaires{region}.pdf")

    fig.show()


## Section NAF

In [ ]:
data_region2019 = data_region.loc[data_region['year'] == 2019.0]
data_region2019_section = data_region2019.groupby('section NAF').sum()
data_region2019_section = data_region2019_section.reset_index()

In [ ]:

fig = px.bar(
    data_region2019_section, 
    x="montant", 
    y='section NAF', 
    barmode="group",  
    height=600,  
    title=f"Montant des subventions par Intitulé NAF section, 2019 {region}",
    orientation='h'  
)

fig.update_xaxes(title_text="Montant")
fig.update_yaxes(title_text="Intitulé Section NAF")

fig.show()
#fig.write_image("Montant_Subvention_code_NAF.pdf")


In [ ]:
data_region2019 = data_region.loc[data_region['year'] == 2019.0]
data_region2019_sub = data_region2019.groupby('Naf_subsector_name').sum()
data_region2019_sub = data_region2019_sub.reset_index()

In [ ]:

fig = px.bar(
    data_region2019_sub, 
    x="montant", 
    y='Naf_subsector_name', 
    barmode="group",  
    height=2000,  
    title=f"Montant des subventions par Intitulé NAF section, 2019 {region}",
    orientation='h'  
)

fig.update_xaxes(title_text="Montant")
fig.update_yaxes(title_text="Intitulé Section NAF sub-sector")

fig.show()
#fig.write_image("Montant_Subvention_code_NAF.pdf")

In [ ]:
data_region2019 = data_region.loc[data_region['year'] == 2019.0]
data_region2019_subsub = data_region2019.groupby('Naf_subsubsector_name').sum()
data_region2019_subsub = data_region2019_subsub.reset_index()

In [ ]:

fig = px.bar(
    data_region2019_subsub, 
    x="montant", 
    y='Naf_subsubsector_name', 
    barmode="group",  
    height=6000,  
    title=f"Montant des subventions par Intitulé NAF section, 2019 {region}",
    orientation='h'  
)

fig.update_xaxes(title_text="Montant")
fig.update_yaxes(title_text="Intitulé Section NAF sub-sub sector")

fig.show()
#fig.write_image("Montant_Subvention_code_NAF.pdf")

# Subvention donné par la région à des communes

In [ ]:
data_administration_gene = data_region.loc[data_region['Naf_subsubsector_name'] == 'Administration publique générale']

Il manque les localisations des communes qui recoivent (toutes ne sont pas dans le csv selected_communities) trouvé ici : https://www.data.gouv.fr/fr/datasets/communes-de-france-base-des-codes-postaux/

In [ ]:
communesdepartementregion = pd.read_csv('data/20230823-communes-departement-region.csv')

In [ ]:
convert_to_int = lambda value: int(float(value)) if isinstance(value, (int, float, str)) and str(value).replace('.', '', 1).isdigit() else value

communes_data = all_communities_data[all_communities_data['type'] == 'COM']
communes_data['cog'] = communes_data['cog'].apply(convert_to_int)

communesdepartementregion['code_commune_INSEE'] = communesdepartementregion['code_commune_INSEE'].apply(convert_to_int)

merge_data = communes_data.merge(
    communesdepartementregion[['code_commune_INSEE', 'latitude', 'longitude']], 
    left_on="cog", right_on="code_commune_INSEE", how="left"
).drop_duplicates()

merge_data = merge_data.rename(columns={'latitude': 'latitude_bene', 'longitude': 'longitude_bene'})
merge_data['siren'] = merge_data['siren'].astype(str)

data_administration_gene2019 = data_administration_gene.merge(
    merge_data[['latitude_bene', 'longitude_bene', 'siren', 'cog']], 
    left_on="siren_bene", right_on="siren", how="left"
).drop_duplicates()

# Filtrage pour l'année 2019
data_administration_gene2019 = data_administration_gene2019[data_administration_gene2019['year'] == 2019]
data_administration_gene2019 = data_administration_gene2019.reset_index(drop=True)
data_administration_gene2019 = data_administration_gene2019.drop_duplicates()


In [ ]:

# Fonction pour formater les montants (Mi pour millions, Md pour milliards)
def custom_format(val):
    if val >= 1_000_000_000:
        return f"{val / 1_000_000_000:.1f} Md"
    elif val >= 1_000_000:
        return f"{val / 1_000_000:.1f} Mi"
    else:
        return f"{val:.0f}"

data_administration_gene2019['montant_formatted'] = data_administration_gene2019['montant'].apply(custom_format)

fig = px.scatter_mapbox(
    data_administration_gene2019,
    lat="latitude_bene", 
    lon="longitude_bene",  
    hover_name="nombeneficiaire",  
    hover_data=["montant_formatted"], 
    color="montant",  
    color_continuous_scale="Viridis",  
    size_max=4.5,  
    title=f"Carte des Subventions donner aux communes par la {region}",
    mapbox_style="carto-positron", 
    zoom=6,  
    range_color=[0, 200000] 
)

fig.update_layout(
    coloraxis_colorbar=dict(
        title="Montant des Subventions (€)",
        tickvals=[0, 10000, 20000,50000,70000,100000,200000],
        ticktext=["0", "10k", "20k", "50k","70k","100k","200k"]
    ),
    width=800, 
    height=600  
)

app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.Graph(id='map', figure=fig),
    html.Div(id='info-box', children="Cliquez sur une commune pour afficher ses informations.")
])

@app.callback(
    Output('info-box', 'children'),
    [Input('map', 'clickData')]
)
def update_info(clickData):
    if clickData:
        commune_name = clickData['points'][0]['hovertext']
        
        commune_data = data_administration_gene2019[data_administration_gene2019['nombeneficiaire'] == commune_name]
        
        objets = commune_data['objet'].unique()
        
        formatted_objets = ", ".join(objets) if len(objets) > 1 else objets[0]
        result = f"Commune: {commune_name}\nObjets associés: {formatted_objets}"

        return result
    else:
        return "Cliquez sur une commune pour afficher ses informations."

if __name__ == '__main__':
    app.run_server(debug=True)


# DEPARTEMENTS